## Programming for Data Analysis - Project 2 ##

**Name: James McEneaney** <br/><br/>
**Course: Higher Diploma in Computing in Data Analytics, ATU Ireland** <br/><br/> 
**Semester: Semester 2 2023** <br/><br/>


***

### Introduction ###

### Data-cleansing ###

To begin, I will download the libraries I will use in this project. I will use pandas to create the dataframes which I will use to analyse the historical climate data:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as ss 

I have downloaded multiple climate related datasets online. These datasets were as follows:

- Atmospheric **carbon dioxide** concentratons from a report by the Intergovernmental Panel on Climate Change (IPCC); most recent year in dataset is 2001.

- Atmospheric **carbon dioxide** concentrations from a 2008 paper in Nature by Luthi et al; most recent year in dataset is 1870 (137 years prior to 2007).

- Atmospheric **carbon dioxide** data from the Mauna Loa Observatory in Hawaii, providing annual data from 1959 to 2022, when precise measurements of atmospheric carbon dioxide first began to be taken. Previously, the analyser tool worked using infrared absorption, but since 2019 it has worked on the basis of a technique called 'Cavity Ring-Down Spectroscopy'. Measurements are assured to be reflective of the true composition of the global atmosophere, due to the location of the observatory and the rigour with which measurements are calibrated and independently verified.

- **Temperature** estimates going back over 820,000 years based on the EPICA Dome C Ice Core Deuterium Data. EPICA stands for European Project for Ice Coring in Antarctica. It estimates past temperature using deuterium as a proxy. Deuterium is a heavy form of hydrogen (containing a neutron in addition to the one proton found in "light" hydrogen). In colder periods, there tends to be less deuterium in ice cores than in warmer periods. The reason for this is that during warm periods, more of the lighter form of hydrogen is evaporated away from the surface of the ocean, so that when the moist air moves northwards to the poles, it contains a higher ratio of heavy hydrogen than it contains during cold periods; precipitation then contains more of this heavy hydrogen and this ultimately shows up in the ice cores. I will refer to this dataset as the "Jouzel" dataset for convenience (Jean Jouzel is a leading glaciologist and climatologist who is one of the creators of the dataset).

- Global **temperature anomoly** data from the National Oceanic and Atmospheric Administration (NOAA). Measurements for the anomoly are taken from 1850 to 2022 and are calculated against the average temperature for the period 1901 to 2000 and are measured in degrees celsius.

- Antarctic **temperature anomoly** data from the National Oceanic and Atmospheric Administration (NOAA). Measurements for the anomoly are taken from 1850 to 2022 and are calculated against the average temperature for the period 1910 to 2000 and are measured in degrees celsius.

- Atmospheric **methane** concentrations estimated from ice core data from Antarctica EPICA Dome C and Law Dome. This covers the period from 797446 years ago to the year 1980. The record until around 1004 AD relates entirely to Dome C; the record between around 1004 AD and 1824 AD relates to a combination of Dome C and Law Dome, while the record from 1824 onwards relates almost entirely to Law Dome. I cut the relevant data from a source csv file from the Environmental Protection Agency website and saved it into my working directory; I will merge and manipulate the data further using python. Figures for methane are measured in parts per billion by volume.

- Recent global atmospheric **methane** concentrations obtained from the NOAA website. Atmospheric methane concentrations are determined from marine surface sites. This dataset covers the period 1984 to 2022.

With the exception of the Mauna Loa dataset, these datasets were not in csv format initially. To make it easier to create dataframes using pandas, I saved all my source files as csv files in my working directory. I will edit the dataframes using python before I analyse the data; this will include renaming column headings to make them easier to work with. I will also add extra columns to datasets to standardise the figures for time measurement and have the oldest years listed first, so my plots print from oldest years at the left to most recent years at the right.

Firstly, I will load up and amend the IPCC CO2 dataset:

In [ ]:
df_co2_ipcc = pd.read_csv("CO2_ipcc_csv.csv")

# rename column names of existing dataframe (setting 'inplace' parameter equal to 'True')
df_co2_ipcc.rename(columns={'Gasage (yr BP) ': 'Year before 1950', 'CO2 (ppmv)': 'CO2', 'sigma mean CO2 (ppmv)': 'sigma mean CO2'}, inplace=True) 

# here I am adding a new column called 'Years' to standardise the time measurements across different datasets.
# I am adding 73 years to every value in the 'years before 1950' column to effectively convert the column to 'years before 2023'. 
# I am then multiplying each year by minus 1, so that the column represents years "in the past"
df_co2_ipcc['Years'] = (df_co2_ipcc['Year before 1950'] + 73) * -1

# reversing the order of the dataset, to for the older years to appear on the left-hand side of the x-axis
df_co2_ipcc = df_co2_ipcc[::-1]

print(df_co2_ipcc)

Next, I will load up and amend the 2008 Nature paper dataset:

In [ ]:
df_co2_nature = pd.read_csv("CO2_nature_csv.csv")

df_co2_nature.rename(columns={'EDC3_gas_a (yr)': 'Years ago', 'CO2 (ppmv)': 'CO2'}, inplace=True)

# Since the paper was written in 2007 and the most recent year in the associated dataset is 137 year prior, I have added 16
# years to the figure for "years ago" when creating the new column 'Years'
df_co2_nature['Years'] = (df_co2_nature['Years ago'] + 16) * -1

df_co2_nature = df_co2_nature[::-1]

print(df_co2_nature)

Next I will load up the relatively small dataset from Mauna Loa, containing data for atmospheric CO2 levels:

In [ ]:
df_mauna_loa_csv = pd.read_csv("mauna_loa_csv.csv")

# I will rename the columns to standardise the heading for the CO2 data:
df_mauna_loa_csv.rename(columns={'year': 'year', 'mean': 'CO2', 'unc':'unc'}, inplace=True)

# I will create a new column called 'Years', to standardise the measurement of time across the datasets.
# I don't need to reverse the rows in this dataset since the earliest year is already in the first row.
df_mauna_loa_csv['Years'] = (2023 - df_mauna_loa_csv['year']) * -1

print(df_mauna_loa_csv)

Next, the "Jouzel" temperature data inferred from deuterium levels in ice cores from EPICA Dome C:

In [ ]:
df_temp_jouzel_csv = pd.read_csv("temp_jouzel_csv.csv")

df_temp_jouzel_csv.rename(columns={'bag': 'bag', 'ztop': 'depth', 'EDC3béta': 'Years before 1950', 'AICC2012': 'year_new', 'deutfinal': 'deuterium', 'temp': 'temp_Kelvin', 'acc-EDC3beta' : 'acc-EDC3beta'}, inplace=True)

# adding a new column called 'Years' to standardise the measurement of time across datasets. 
# this represents the number of years ago the data relates to eg. -55 relates to (-55 + 73 = 18 years ago, or the year 2005)
# The negative figure allows for my graph to print from oldest date to most recent
df_temp_jouzel_csv['Years'] = (df_temp_jouzel_csv['year_new'] + 73) * -1

# adding a new column called 'temp_Celsius' as I am more familiar with this unit of measurement for temperature
df_temp_jouzel_csv['temp_Celsius'] = (df_temp_jouzel_csv['temp_Kelvin'] - 273.15)

df_temp_jouzel_csv = df_temp_jouzel_csv[::-1]

print(df_temp_jouzel_csv)

I will now load up the dataset for the Antarctic temperature anomaly and create new column which standardises the data for the year to allow for easier comparison with other datasets:

In [ ]:
df_antarctic_temp_anomaly_1850 = pd.read_csv("antarctic_temp_anomaly_csv.csv")

# renaming columns to standardise across datasets when combining later on
df_antarctic_temp_anomaly_1850.rename(columns={'Year': 'year', 'Anomaly': 'anomaly'}, inplace=True)

# adding a new column called 'Years' to standardise the measurement of time across datasets. 
# this represents the number of years ago the data relates to eg. - 1 is 1 year ago ie. 2022
df_antarctic_temp_anomaly_1850['Years'] = (df_antarctic_temp_anomaly_1850['year'] - 2023)

print(df_antarctic_temp_anomaly_1850)


I will also upload the data for Global temperature anomoly from 1850 to 2022, and add a column for 'years ago' as I did for the antarctic temperature anomaly data:

In [ ]:
df_global_temp_anomaly_1850 = pd.read_csv("global_temp_anomaly_csv.csv")

# renaming columns to standardise across datasets when combining later on
df_global_temp_anomaly_1850.rename(columns={'Year': 'year', 'Anomaly': 'anomaly'}, inplace=True)

# adding a new column called 'Years' to standardise the measurement of time across datasets. 
# this represents the number of years ago the data relates to eg. - 1 is 1 year ago ie. 2022
df_global_temp_anomaly_1850['Years'] = (df_global_temp_anomaly_1850['year'] - 2023)

print(df_global_temp_anomaly_1850)

I will now create dataframes for the data relating to methane and modify them where necessary. Firstly, the paleo data:

In [26]:
df_ch4_paleo = pd.read_csv("ch4_paleo_csv.csv")

# Here I am adding a new column called 'Years' to standardise the time measurements across different datasets.
# The original dataset split the years by AD/BC so I subtracted 2023 from each year to standardise the measurement of time.
df_ch4_paleo['Years'] = (df_ch4_paleo['year'] - 2023)

# Here I am merging two columns using the combine_first method: this creates a new column where missing/ NaN values
# from one column are replaced with the corresponding non-null value from the other column. 
# This is to create just one column of paleohistorical data for atmospheric methane concentration.
df_ch4_paleo['CH4'] = df_ch4_paleo['dome_c'].combine_first(df_ch4_paleo['law_dome'])

# Next I am dropping the unwanted columns 'dome_c', 'law_dome' and 'year'. This gives us
# a dataframe containing only years and atmospheric CH4 concentrations up until 1980.
# The axis = 1 parameter is used to specify that it is a column being dropped as opposed to a row
df_ch4_paleo = df_ch4_paleo.drop(['dome_c', 'law_dome', 'year'], axis = 1)

print(df_ch4_paleo)

         Years       CH4
0    -799469.1     399.3
1    -798490.4     457.8
2    -797350.0     396.1
3    -796392.7     418.0
4    -795011.0     428.3
...        ...       ...
2170     -50.0  1,385.80
2171     -49.0  1,421.50
2172     -48.0  1,420.40
2173     -46.0  1,458.10
2174     -43.0  1,479.50

[2175 rows x 2 columns]


Secondly, the more recent methane data:

In [33]:
df_ch4_recent = pd.read_csv("ch4_recent_csv.csv")

# Here I am adding a new column called 'Years' to standardise the time measurements across different datasets.
# The original dataset split the years by AD/BC so I subtracted 2023 from each year to standardise the measurement of time.
df_ch4_recent['Years'] = (df_ch4_recent['year'] - 2023)

# I am converting the figures in the years columns to float to avoid getting a warning message later when I try to 
# combine the 'Years' data from the recent data (which is in integer format) with the 'Years' data from the 
# paleo dataset, which is in float format
df_ch4_recent['Years'] = df_ch4_recent['Years'].astype(float)

# Next I am dropping the unwanted column 'year'. This gives us a dataframe containing only years 
# and atmospheric CH4 concentrations from 1984 to 2022.
# The axis = 1 parameter is used to specify that it is a column being dropped as opposed to a row
df_ch4_recent = df_ch4_recent.drop(['year'], axis = 1)

print(df_ch4_recent)

        CH4  Years
0   1644.85  -39.0
1   1657.29  -38.0
2   1670.09  -37.0
3   1682.70  -36.0
4   1693.28  -35.0
5   1704.53  -34.0
6   1714.43  -33.0
7   1724.78  -32.0
8   1735.47  -31.0
9   1736.53  -30.0
10  1742.07  -29.0
11  1748.88  -28.0
12  1751.28  -27.0
13  1754.48  -26.0
14  1765.54  -25.0
15  1772.33  -24.0
16  1773.22  -23.0
17  1771.22  -22.0
18  1772.66  -21.0
19  1777.33  -20.0
20  1777.05  -19.0
21  1774.16  -18.0
22  1774.94  -17.0
23  1781.38  -16.0
24  1787.01  -15.0
25  1793.52  -14.0
26  1798.91  -13.0
27  1803.14  -12.0
28  1808.12  -11.0
29  1813.43  -10.0
30  1822.53   -9.0
31  1834.26   -8.0
32  1843.12   -7.0
33  1849.58   -6.0
34  1857.33   -5.0
35  1866.58   -4.0
36  1878.93   -3.0
37  1895.28   -2.0
38  1911.83   -1.0


Finally, we can merge the paleo and recent methane datasets to enable us to plot it later on:

In [36]:
df_ch4 = pd.merge(df_ch4_paleo, df_ch4_recent, on = 'Years', how = 'outer')

# The merge produces two columns: 'CH4_x' relating to the paleo data and 'CH4_y' relating to the recent data.
# Here I am merging these two columns using the combine_first method: this creates a new column where missing/ NaN values
# from one column are replaced with the corresponding non-null value from the other column

df_ch4['CH4'] = df_ch4['CH4_x'].combine_first(df_ch4['CH4_y'])

# Here I am dropping the unwanted columns CH4_x and CH4_y. 
# The axis = 1 parameter is used to specify that it is a column being dropped as opposed to a row
df_ch4 = df_ch4.drop(['CH4_x', 'CH4_y'], axis = 1)

print(df_ch4)

         Years      CH4
0    -799469.1    399.3
1    -798490.4    457.8
2    -797350.0    396.1
3    -796392.7    418.0
4    -795011.0    428.3
...        ...      ...
2209      -5.0  1857.33
2210      -4.0  1866.58
2211      -3.0  1878.93
2212      -2.0  1895.28
2213      -1.0  1911.83

[2214 rows x 2 columns]


#### Plots of datasets ####

*IPCC data:*

In [ ]:
plt.figure(figsize= (16,6))
plt.title('Atmospheric carbon dioxide concentrations (parts per million by volume) over the past 800000 years - IPCC data')

sns.scatterplot(x ='Years', y = 'CO2', data = df_co2_ipcc, size = 40)      # use 'size' parameter to decrease size of markers 

sns.lineplot(x ='Years', y = 'CO2', data = df_co2_ipcc, color = 'blue')

*Nature data:*

In [ ]:
plt.figure(figsize= (16,6))
plt.title('Atmospheric carbon dioxide concentrations (parts per million by volume) over the past 800000 years - Nature data')

sns.scatterplot(x='Years', y='CO2', data=df_co2_nature, color = 'orange', size = 40)      # use 'size' parameter to decrease size of markers 

sns.lineplot(x = 'Years', y = 'CO2', data = df_co2_nature, color = 'orange')


*Mauna Loa:*

In [ ]:
plt.figure(figsize= (16,6))
plt.title('Atmospheric carbon dioxide concentrations (parts per million by volume) since 1959 - Mauna Loa data')

# Here I will plot using the column 'year' as opposed to 'Years', as the graph for recent data is easier to comprehend 
# when the x-axis is labelled by calendar year rather than how many years ago the year was
# We use the 'size' parameter to decrease size of markers 
sns.scatterplot(x = 'year', y = 'CO2', data=df_mauna_loa_csv, color = 'red', size = 20)      

sns.lineplot(x = 'year', y = 'CO2', data = df_mauna_loa_csv, color = 'red')

*Jouzel data:*

In [ ]:
plt.figure(figsize= (16,6))
plt.title('Temperature estimates for past 820000 years inferred from deuterium measurements in EPICA Dome C ice cores')
plt.xlabel('Years')
plt.ylabel('Temperature (°C)')

sns.scatterplot(x = 'Years', y = 'temp_Celsius', data=df_temp_jouzel_csv, color = 'green', size = 20)      

sns.lineplot(x = 'Years', y = 'temp_Celsius', data = df_temp_jouzel_csv, color = 'green')


#### Comparison of C02 datasets ####

To get a sense of the extent to which carbon dioxide levels may have increased in recent years compared to the long-term historical trends, I would ideally have a dataset which contains carbon-dioxide data from the distant past right up to the present day. 

Fortunately, I can obtain this by combining the data from the IPCC dataset with the more recent data from the Mauna Loa dataset. 

Firstly, I want give myself some confidence that the long-term historical data in the IPCC dataset is valid; to do this, I will compare it with the 2008 Nature dataset by creating plot a lineplot for each dataset on the same graph:

In [ ]:
plt.figure(figsize= (16,6))

sns.lineplot(x='Years', y='CO2', data=df_co2_ipcc, color = 'blue', label='IPCC')
sns.lineplot(x='Years', y='CO2', data=df_co2_nature, color = 'orange', label='Nature')

plt.title('Atmospheric CO2 over 800000 years - IPCC vs Nature data')
plt.xlabel('Years')
plt.ylabel('CO2 Concentration (ppmv)')

plt.legend()

plt.show()

plt.savefig('C02_IPCC_vs_Nature.png')

We see that the datasets are largely in agreement with one another. The Nature data ends in 1870, while the IPCC data continues on to 2001, and the large spike in carbon dioxide levels seen on the right hand-side of the graph relates to these additional years present in the IPCC data.

We can also plot the most recent carbon dioxide data from the IPCC dataset with it's counterpart data from the Mauna Loa dataset:

In [ ]:
# create a new sub-dataframe from the IPCC dataframe containing only rows relating to 1959 onwards 
#(ie. up to last data point of 2001)
df_co2_ipcc_1959_to_2001 = df_co2_ipcc.loc[df_co2_ipcc['Years'] >= -64]

# create a new sub-dataframe from the IPCC dataframe containing only rows relating to 2001 backwards 
# (ie. back to first data point of 1959)
df_mauna_loa_1959_to_2001 = df_mauna_loa_csv.loc[df_mauna_loa_csv['Years'] <= -22]

#print(df_co2_ipcc_1959_to_2001)

#print(df_mauna_loa_1959_to_2001)

Now we can plot the data from these two sub-dataframes against each other:

In [ ]:
plt.figure(figsize= (16,6))

sns.lineplot(x='Years', y='CO2', data=df_co2_ipcc_1959_to_2001, color = 'blue', label='IPCC')
sns.lineplot(x='Years', y='CO2', data=df_mauna_loa_1959_to_2001, color = 'red', label='Mauna Loa')

plt.title('Atmospheric CO2 between 1959 and 2001 - IPCC vs Mauna Loa data')
plt.xlabel('Years ago')
plt.ylabel('CO2 Concentration (ppmv)')

plt.legend()

plt.show()

plt.savefig('co2_ipcc_vs_mauna_loa.png')

It should be noted that the IPCC data relates to CO2 levels inferred from high resolution ice cores in the Law Dome in East Antarctica while the Mauna Loa data comes from the summit of Mauna Loa in Hawaii, using a sophisticated analyser tool. Nevertheless, we can see that the data for CO2 concentration during the overlapping period of the IPCC and Mauna Loa datasets is in agreement.

#### Combining IPCC and Mauna Loa Carbon Dioxide datasets ####

I will now combine the IPCC data with the Mauna Loa data to create a new dataframe containing data for carbon dioxide levels from 805,741 years ago to today:

In [ ]:
# First I will create a sub-dataframe from the Mauna Loa containing CO2 data relating to 2002 onwards
df_post_2001_mauna_loa = df_mauna_loa_csv.loc[df_mauna_loa_csv['Years'] > -22 ]

# creating a new dataframe from the Mauna Loa data, containing only the two columns of interest 'CO2' and 'Years'.
# To avoid getting a warning for 'chained indexing', I followed the syntax as found in this link : 
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
df_post_2001_mauna_loa_trimmed = df_post_2001_mauna_loa.loc[:, ('Years', 'CO2')]

# I am converting the figures in the years columns to float to avoid getting a warning message when I combine the 'Years' data
# from the Mauna Loa data (which is in integer format) with the 'Years' data from the IPCC dataset, which is in float format
df_post_2001_mauna_loa_trimmed['Years'] = df_post_2001_mauna_loa_trimmed['Years'].astype(float)

#print(df_post_2001_mauna_loa_trimmed)

# creating a new dataframe from the IPCC data, containing only the two columns of interest 'CO2' and 'Years'
df_co2_ipcc_trimmed = df_co2_ipcc.loc[:, ('Years', 'CO2')]

#print(df_co2_ipcc_trimmed)

# Here I am merging the trimmed dataframes for the IPCC data and post-2001 Mauna Loa data 
df_co2 = pd.merge(df_co2_ipcc_trimmed, df_post_2001_mauna_loa_trimmed, on = 'Years', how = 'outer')

# The merge produces two columns: 'C02_x' relating to the IPCC data and 'CO2_y' relating to the Manua Loa data.
# Here I am merging these two columns using the combine_first method: this creates a new column where missing/ NaN values
# from one column are replaced with the corresponding non-null value from the other column

df_co2['CO2'] = df_co2['CO2_x'].combine_first(df_co2['CO2_y'])

# Here I am dropping the unwanted columns CO2_x and CO2_y. the axis = 1 parameter is used to specify that it is a column being
# dropped as opposed to a row
df_co2 = df_co2.drop(['CO2_x', 'CO2_y'], axis = 1)

print(df_co2)

Next, I will plot this composite carbon dioxide dataset to show how the most recent figures for carbon dioxide levels compare with the long-term historical average:

In [ ]:
plt.figure(figsize= (16,6))
plt.title('Atmospheric carbon dioxide concentrations (parts per million by volume) over the past 800000 years')

sns.scatterplot(x ='Years', y ='CO2', data=df_co2, color ='orange', size = 20)      # use 'size' parameter to decrease size of markers 

sns.lineplot(x = 'Years', y = 'CO2', data = df_co2, color = 'orange')

We can see clearly that atmospheric carbon dioxide levels have risen greatly in a very short period of time to reach levels significantly higher than the levels which prevailed for the past 800000 years.

### CO2 versus Temperature Anomoly ###

To compare CO2 levels for the past 820000 years with the temperature anomaly, I will need to obtain a dataset for the temperature anomaly covering this whole timespan. We can use data from the NOAA for the Antarctic air temperature anomaly for the period from 1850 to now. This record compares the temperature for each year to the average temperature of the base period 1910 to 2000. 

For earlier data, I will calculate the temperature anomaly as follows:
I will take the average temperature for the period 1000 AD to 1845 AD from the Jouzel dataset, calculated based on the number of data points for temperature during this timespan; I am choosing 1845 AD since this is the most recent figure for temperature given in the Jouzel dataset which doesn't have a counterpart datapoint in the NOAA data. The reason I am using almost 1000 years of data as a base period is that this is the base period used in the earlier Jouzel dataset (found on the text file here: https://www1.ncdc.noaa.gov/pub/data/paleo/icecore/antarctica/epica_domec/edc3deuttemp2007.txt). I wasn't able to easily extract this temperature anomaly data from the text file so I decided to calculate it myself based on the newer dataset (saved in my working directory as a csv file).

Since 1845 is a relatively early date where humanity was yet to release large amounts of carbon dioxide into the atmosphere, it seems acceptable to me to compare the temperature in 1845 to the pre-industrial temperatures of earlier times. 

I will then create a dataframe containing the temperature anomaly going back to 820000 years ago, right up to the present day. I will plot these figures against the figures for CO2 obtained above.

In [ ]:
# First I will create a sub-dataframe from the Jouzel dataset containing temperature data relating to 1845 backwards
df_pre_1845_jouzel = df_temp_jouzel_csv.loc[df_temp_jouzel_csv['Years'] < -177 ]

# creating a new dataframe from the Jouzel sub-dataframe, containing only the two columns of interest 'Years' and 'temp_Kelvin'
# To avoid getting a warning for 'chained indexing', I followed the syntax as found in this link : 
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
df_pre_1845_jouzel_trimmed = df_pre_1845_jouzel.loc[:, ('Years', 'temp_Kelvin')]

# I am converting the figures in the years columns to float to avoid getting a warning messages if I combine data relating to
# years from different datasets
df_pre_1845_jouzel_trimmed['Years'] = df_pre_1845_jouzel_trimmed['Years'].astype(float)

# to calculate the average temperature between 1000 AD and 1845 AD, we only take rows relating to 1013 years ago and onwards
df_1000_to_1845_jouzel = df_pre_1845_jouzel_trimmed.loc[df_pre_1845_jouzel_trimmed['Years'] > - 1013]

mean_temp_1000_to_1845 = df_1000_to_1845_jouzel['temp_Kelvin'].mean()
round_mean_temp_1000_to_1845 = round(mean_temp_1000_to_1845, 2)
print(f"The mean Antarctic air temperature between 1000 AD and 1845 AD is estimated to be {round_mean_temp_1000_to_1845} Kelvin \n")

# Next I want to calculate the temperature anomoly for each year in the Jouzel dataset using this base period temperature of 217.18 kelvin:
df_pre_1845_jouzel_trimmed['anomaly'] = df_pre_1845_jouzel_trimmed['temp_Kelvin'] - 217.18
print("The temperature anomaly in degrees compared with the average for the period 1000 to 1845 is as follows: \n ")
print(df_pre_1845_jouzel_trimmed)

# Next I want to combine this pre 1845 temperature anomaly data with the post 1850 darta from the NOAA:
# Firstly I will ensure the data for 'Years' in the NOAA daaset is in float format, to avoid getting a warning
# for trying to merge floats and ints in the same column
df_antarctic_temp_anomaly_1850['Years'] = df_antarctic_temp_anomaly_1850['Years'].astype(float)

df_temp_anomaly = pd.merge(df_pre_1845_jouzel_trimmed, df_antarctic_temp_anomaly_1850, on = 'Years', how = 'outer')

# The merge produces two anomaly columns: 'anomaly_x' relating to the Jouzel data and 'anomaly_y' relating to the NOAA data.
# Below I am merging these two columns using the combine_first method: this creates a new column where missing/ NaN values
# from one column are replaced with the corresponding non-null value from the other column

df_temp_anomaly['anomaly'] = df_temp_anomaly['anomaly_x'].combine_first(df_temp_anomaly['anomaly_y'])

# Next I am dropping the unwanted columns 'anomaly_x', 'anomaly_y', 'year' and 'temp_Kelvin'. This gives us
# a dataframe containing only years and the temperature anomly for each year in degrees (Celsius or Kelvin).
# The axis = 1 parameter is used to specify that it is a column being dropped as opposed to a row
df_temp_anomaly = df_temp_anomaly.drop(['anomaly_x', 'anomaly_y', 'year', 'temp_Kelvin'], axis = 1)
                                    
print(df_temp_anomaly)

#### CO2 versus temperature anomaly ####

We can now plot the composite dataset for CO2 levels against the composite dataset for the temperature anomoly, over approximately 820000 years:

In [ ]:
# This creates a plot capable of displaying more than one subplot
# https://python-graph-gallery.com/line-chart-dual-y-axis-with-matplotlib/#:~:text=It%20uses%20ax.,Y%20axis%20on%20this%20twin.
# Below is an alteration of some code I used in Project 1 for the Programming for Data Analysis module of my HDip in
# Data Analytics https://github.com/jmce22/pfda_project1 
# In that project I created a graph with two y-axes but with the y-axes data measured against the same x-axis

fig, ax1 = plt.subplots(figsize=(16, 6))

# This creates a second y-axis which shares the same x-axis as the first y-axis
ax2 = ax1.twinx()

# Plot for CO2 on the primary y-axis
ax1.plot(df_co2['Years'], df_co2['CO2'], color='blue', label='CO2')
ax1.set_xlabel('Years')
ax1.set_ylabel('CO2', color='blue')
ax1.tick_params(axis='y', colors='blue')   

# Plot for temperature on secondary y-axis
ax2.plot(df_temp_anomaly['Years'], df_temp_anomaly['anomaly'], color='red', label='Temperature')
ax2.set_ylabel('Temperature (°C)', color='red')
ax2.tick_params(axis='y', colors='red')

plt.title('CO2 versus temperature over the past 820000 years') 
plt.grid(True)    # This plots a grid which makes it easier to compare both sets of data
plt.savefig('CO2_vs_temp')
plt.show()

It seems to be the case that over the past 820000 years, the temperature anomoly was associated with the level of CO2 in the atmosphere, as local peaks in the data for CO2 tend to occur during the same periods as local peaks in the air temperature data. 

To compare the data for the Antarctic temperature anomaly from 1850 with the figures for CO2 from 1850, I can create a sub-dataframe for the CO2 for 1850 to 2022 and plot it in the same way I plotted the full datasets; by doing this, we can investigate the temperature anomoly as it relates to recent increases in CO2 levels in the atmosphere. It should be borne in mind that the figures for temperature anomoly for 1850 onwards are calculated based on a the average Antarctic air temperature for the period 1910 to 2000.


In [ ]:
df_co2_post_1850 = df_co2.loc[df_co2['Years'] > -174]
print(df_co2_post_1850)

# I will also create a sub-frame from the composite temperature anomaly dataset to include only years from 1850 onwards,
# as I want to use the 'Years' column which has a standardised format for measuring time
df_temp_anomaly_post_1850 = df_temp_anomaly.loc[df_temp_anomaly['Years'] > -174]
print(df_temp_anomaly_post_1850)

We can plot this CO2 data against the temperature anomaly data for the same time range:

In [ ]:
fig, ax1 = plt.subplots(figsize=(16, 6))

# This creates a second y-axis which shares the same x-axis as the first y-axis
ax2 = ax1.twinx()

# Plot for CO2 on the primary y-axis
ax1.plot(df_co2_post_1850['Years'], df_co2_post_1850['CO2'], color='blue', label='CO2')
ax1.set_xlabel('Years')
ax1.set_ylabel('CO2', color='blue')
ax1.tick_params(axis='y', colors='blue')   

# Plot for temperature anomaly on secondary y-axis. 
ax2.plot(df_temp_anomaly_post_1850['Years'], df_antarctic_temp_anomaly_1850['anomaly'], color='red', label='Temperature')
ax2.set_ylabel('Temperature Anomaly(°C)', color='red')
ax2.tick_params(axis='y', colors='red')

plt.title('Antarctic Atmospheric CO2 levels versus Antarctic temperature anomaly from 1850 to 2022') 
plt.grid(True)    # This plots a grid which makes it easier to compare both sets of data
plt.savefig('CO2_vs_temp_anomaly')
plt.show()

When we plot these two variables together, we can see that the data can be divided into two sections:

- The data from 1850 to 1950 shows that temperature didn't fluctuate greatly and was typically lower than the average for 1910 to 2000 as a whole. Carbon dioxide levels also didn't increase hugely over this time period relative to the increases which come after that point (levels increased from around 286 ppm to 313 ppm between 1850 and 1950 according to the IPCC data: an increase of around 9.4% over a century)

- The data from 1950 to 2022 shows that temperatures fluctuated a lot more compared to the average for 1910 to 2000 and that for most of the period since around 1965, average temperature at the Antarctic has been higher than this base figure. One interesting point I notice is that the temperature anomaly reached levels of over 0.5 degrees above baseline during four years from 1988 to 2007, but only surpassed 0.4 degrees above baseline twice since 2007; an interesting investigation might be to compare levels of CFCs released into the atmosphere versus the temperature anomaly, as the use of CFCs began to be banned from the late 1980s onwards when it was recognised that they played a huge role destroying the ozone layer and in causing global warming.

A manual investigation of the IPCC dataset shows that it took around **53 years** (from 1912 to 1965) for CO2 levels to go from 300 to 320 ppm. According to the Mauna Loa dataset, it then took only **16 years** to increase to 340 ppm (in 1981), **14 years** to reach 360 ppm (in 1995), **10 years** to reach 380 ppm (in 2005) and **10 years** to reach 400 ppm in 2015. The most recent figures for the year 2022 are 418 ppm.

### CH4 versus Temperature Anomoly ###

Methane is a potent greenhouse gas and the addition of methane into the atmosphere from human economic activity is known to be contributing to atmospheric warming and climate change. It has a 'Global Warming Potential' (GWP) 84 times greater than CO2 (the reference gas used for calculating GWP) over a 20-year period, meaning a tonne of methane would absorb as much infrared thermal radiation as 84 tonnes of CO2; therefore, it's release into the atmosphere is 84 times more potent than an equivilant quantity released of CO2 over that timespan. However, methane has a shorter lifespan in the atmosphere than CO2, so its GWP over much longer timespans (eg. centuries) is lower than its GWP over decades. 

By way of comparison with the 20-year GWP value of 84 for methane, the 20-year GWP figures for Dichlorodifluoromethane (a CFC responsible for depletion of the ozone layer) is 10800, which highlights how it important it was that the manufacture of CFCs such as this was banned globally in 2010 (although recent studies have found that some CFCs are still being produced illegally in China).



### Climate change signals in the Irish context ###

### Fusion of datasets ###

### Prediction of Global Temperature Anomoly ###

### Accelerating increases of temperature ###

### Summary ###

### References ###